In [1]:
import pandas as pd
import openpyxl

In [2]:
df = pd.read_excel('GENeSYS-MOD_User_Input_Settings_v01_phe_05-09-2023.xlsx', sheet_name='Region_selection')

# Now you can work with the DataFrame (e.g., display, manipulate, analyze data)
print(df)

  Continent Country Country_Sublevel_1 Country_Sublevel_2  Region Selected
0    Europe      DE              DE-BY           DE-BY-UF                1
1    Europe      NO                NO1               NO11                1
2    Europe      NO                NO1                NO1                1
3    Europe      NO                 NO                 NO                0
4    Europe      DE              DE-BY              DE-BY                1
5    Europe      DE                 DE                 DE                1
6      Asia      CN                 CN                 CN                0
7      Asia      CN             CN-Cen             CN-Cen                1
8      Asia      CN             CN-Cen       CN-Cen-Henan                1


In [3]:
filtered_df = df[df.iloc[:, -1] == 1]

# Now 'filtered_df' contains only the entries with a 1 in the last field
print(filtered_df)

  Continent Country Country_Sublevel_1 Country_Sublevel_2  Region Selected
0    Europe      DE              DE-BY           DE-BY-UF                1
1    Europe      NO                NO1               NO11                1
2    Europe      NO                NO1                NO1                1
4    Europe      DE              DE-BY              DE-BY                1
5    Europe      DE                 DE                 DE                1
7      Asia      CN             CN-Cen             CN-Cen                1
8      Asia      CN             CN-Cen       CN-Cen-Henan                1


In [ ]:
# Read the CSV file into a Pandas DataFrame, starting from row 5 and using ';' as the delimiter
second_df = pd.read_csv('csvs\Par_RegionalBaseYearProduction.csv', skiprows=4, delimiter=';')

# Now you have the data from the CSV file stored in the 'second_df' DataFrame
print(second_df)

In [ ]:
# Extract the unique values from the fourth entry (dimension) of the first DataFrame
unique_values_from_first_df = filtered_df.iloc[:, 3].unique()

# Filter the second DataFrame to keep only the rows where the first dimension is in the unique values from the first DataFrame
filtered_second_df = second_df[second_df.iloc[:, 0].isin(unique_values_from_first_df)]

# Now 'filtered_second_df' contains only the values where the first dimension can be found in the fourth entry of the first DataFrame
print(filtered_second_df)

In [ ]:
# Read the CSV file into a Pandas DataFrame, starting from row 5 and using ';' as the delimiter
third_df = pd.read_csv('csvs\Par_CapitalCost.csv', skiprows=4, delimiter=';')

# Now you have the data from the CSV file stored in the 'second_df' DataFrame
print(third_df)

In [ ]:
# Filter the second DataFrame to keep only the rows where the first dimension is in the unique values from the first DataFrame
filtered_third_df = third_df[third_df.iloc[:, 0].isin(unique_values_from_first_df)]

# Now 'filtered_second_df' contains only the values where the first dimension can be found in the fourth entry of the first DataFrame
print(filtered_third_df)

In [ ]:
# Create a Pandas Excel writer
with pd.ExcelWriter('output.xlsx', engine='openpyxl') as writer:
    # Write the first DataFrame to the Excel file
    df.to_excel(writer, sheet_name='FirstDataFrame', index=False)
    
    # Write the second filtered DataFrame to the Excel file
    filtered_second_df.to_excel(writer, sheet_name='FilteredSecondDataFrame', index=False)
    
    # Write the third filtered DataFrame to the Excel file
    filtered_third_df.to_excel(writer, sheet_name='FilteredThirdDataFrame', index=False)
